# General Computations for Differentiating Arbitrary Policies for Rollout Bayesian Optimization
`Learning outcome`: Mapping computations required for solving rollout acquisition functions.

## Introductory Remarks
In this discussion, we will examine the general problem of minimizing an unknown and potentially expensive-to-evaluate function $f(x)$. This is a common challenge in many applications, from hyperparameter tuning in machine learning to optimizing physical experiments. One powerful strategy for addressing this problem is Bayesian optimization, which efficiently searches for the minimum of $f(x)$ by building a probabilistic model that balances exploration of new regions of the search space with exploitation of known promising areas. By leveraging this model, Bayesian optimization can minimize $f(x)$ with fewer evaluations compared to traditional optimization methods.

First, I will introduce myopic Bayesian optimization, which focuses on selecting the next evaluation point based on immediate benefits, maximizing short-term improvements. While effective in many cases, myopic approaches can overlook long-term gains. To address this, we turn to non-myopic Bayesian optimization, which incorporates future consequences of current decisions, leading to more strategic and globally informed exploration.

A key element in non-myopic Bayesian optimization is the use of rollout acquisition functions. These functions balance exploration and exploitation by simulating potential future outcomes of the optimization process, guiding decisions that are expected to yield better results over multiple steps.

By understanding these approaches and their applications, we can enhance our ability to optimize complex systems more effectively and efficiently.

### Useful Notation
$$
\DeclareMathOperator*{\argmax}{arg\,max}
$$
We have some collection of regressors $X \in \mathbb{R}^{d \times n}$ and observations $\mathcal{\textbf{y}} \in \mathbb{R}^{n}$, where our dataset for our supervised machine learning model is denoted as $\mathcal{D}_n = \{(x_i, y_i) : 1 \leq i \leq n\}$. Given $\mathcal{D}_{n}$, we denote the predictive mean and predictive variance at some location $x$ as $\mu_n(x|\mathcal{D}_n)$ and $\sigma_n(x|\mathcal{D}_n)$ respectively. In general, we'll suppress the parameter $\mathcal{D}_n$, and write $\mu_n(x)$ and $\sigma_n(x)$.

## Myopic Bayesian optimization
**Myopic Bayesian Optimization (MBO)** provides a strategy for this problem, typically using a Gaussian Process (GP) to model $f(x)$ based on the data gathered up to the current step, denoted as $D_n$.

In the myopic setting, the next evaluation point $x_{n+1}$ is selected by maximizing an acquisition function $\alpha(x \mid \mu_n(x), \sigma_n(x), \theta)$, which uses the current model to decide where the next sample should be taken. This is done with a short-term perspective, looking only one step ahead:

$$
x_{n+1} = \argmax_{x \in \Omega} \; \alpha(x \mid \mu_n(x), \sigma_n(x), \theta)
$$

The key characteristic of myopic Bayesian optimization is that it makes decisions based on maximizing some immediate reward, depending only on the observations made so far. While this approach is computationally efficient and often effective in practice, it tends to focus on short-term gains, optimizing for the current step without considering the long-term impact on the overall optimization process. Consequently, when we have more than one step remaining in our evaluation budget, myopic acquisition functions may overlook opportunities that would be better explored over the long term. This limitation highlights the need for strategies that account for future consequences, which non-myopic approaches aim to address.

## Non-myopic Bayesian optimization
**Non-myopic Bayesian optimization (NMBO)** frames the exploration-exploitation problem as a balance of immediate and future rewards. Lam et al. formulate NMBO as a finite-horizon dynamic program; the equivalent Markov decision process (MDP) follows.

The notation used is standard in Puterman: an MDP is a collection $(T, \mathbb{S}, \mathbb{A}, P, R)$, where $T = \{1,2,\dots,h\},$ and $h < \infty$ is the set of decision epochs, finite for our problem. The state space, $\mathbb{S}$, encapsulates the information needed to model the system from time $t \in T$, and $\mathbb{A}$ is the action space. Given a state $s \in \mathbb{S}$ and an action $a \in \mathbb{A}$, $P(s'|s,a)$ is the probability that the next state will be $s'$. $R(s,a,s')$ is the reward received for choosing action $a$ in state $s$ and transitioning to $s'$.

A decision rule $\pi_t : \mathbb{S} \to \mathbb{A}$ maps states to actions at time $t$. A policy $\pi$ is a series of decision rules $\pi = (\pi_1, \pi_2, \dots, \pi_h)$, one at each decision epoch. Given a policy $\pi$, a starting state $s_0$, and horizon $h$, we can define the expected total reward $V_h^\pi(s_0)$ as:

$$
  V_h^\pi(s_0) = \mathbb{E}\left[ \sum_{t=0}^{h-1} R(s_t, \pi_t(s_t), s_{t+1}) \right].
$$

Our objective is to find the optimal policy $\pi^*$ that maximizes the expected total reward, i.e., $\sup_{\pi \in \Pi} V_h^\pi(s_0)$, where $\Pi$ is the space of all admissible policies.

If we can sample from the transition probability $P$, we can estimate the expected total reward of any base policy — the decisions made using the base acquisition function — with Monte Carlo (MC) integration:

$$
  V_h^{\hat{\pi}}(s_0) \approx 
  \frac{1}{N} \sum_{i=1}^N \left[ \sum_{t=0}^{h-1} R(s_t^i, \hat{\pi}_t(s_t^i), s_{t+1}^i) \right].
$$

Given a Gaussian Process (GP) prior over data $\mathcal{D}_n$ with mean $\mu_{n}$ and covariance matrix $K_{(n)}$, we model $h$ steps of Bayesian optimization as an MDP. This MDP's state space is all possible data sets reachable from the starting state $\mathcal{D}_n$ with $h$ steps of Bayesian optimization. Its action space is $\Omega$; actions correspond to sampling a point in $\Omega$. Its transition probability and reward function are defined as follows. Given an action $\mathbf{x}_{n+1}$, the transition probability from $\mathcal{D}_n$ to $\mathcal{D}_{n+1}$, where $\mathcal{D}_{n+1} = \mathcal{D}_n \cup \{(\mathbf{x}_{n+1}, y_{n+1})\}$ is:

$$
  P(\mathcal{D}_n, \mathbf{x}_{n+1}, \mathcal{D}_{n+1}) \sim 
  \mathcal{N}(\mu_{n}(\mathbf{x}_{n+1}; \mathcal{D}_n), K_{(n)}(\mathbf{x}_{n+1}, \mathbf{x}_{n+1}; \mathcal{D}_n)).
$$

Thus, the transition probability from $\mathcal{D}_n$ to $\mathcal{D}_{n+1}$ is the probability of sampling $y_{n+1}$ from the posterior $\mathcal{GP}(\mu_{n}, K_{(n)})$ at $\mathbf{x}_{n+1}$. The reward function $R(\mathcal{D}_n, \mathbf{x}_{n+1}, \mathcal{D}_{n+1})$ can be any general reward that reflects the value of transitioning from state $\mathcal{D}_n$ to state $\mathcal{D}_{n+1}$ after sampling $\mathbf{x}_{n+1}$. 

EI (expected improvement) is a special case where the reward is the improvement over the best-observed value, but here we consider a more general reward structure.

We define the non-myopic policy as the optimal solution to an $h$-horizon MDP. The expected total reward of this MDP is:

$$
  V_h^\pi(\mathcal{D}_n) = \mathbb{E}\left[ \sum_{t=n}^{n+h-1} R(\mathcal{D}_t, \pi_t(\mathcal{D}_t), \mathcal{D}_{t+1}) \right].
$$

When $h > 1$, the optimal policy is difficult to compute.

## Rollout Acquisition Functions